In [1]:
import tensorflow as tf
from datetime import datetime

%load_ext tensorboard

## Defining models and layers in TensorFlow

simple module


In [2]:
class SimpleModule(tf.Module):
  def __init__(self, name=None):
    super().__init__(name=name)
    self.a_variable = tf.Variable(5.0, name="train_me")
    self.non_trainable_variable = tf.Variable(5.0, trainable=False, name="do_not_train_me")
  def __call__(self, x):
    return self.a_variable * x + self.non_trainable_variable

simple_module = SimpleModule(name="simple")

simple_module(tf.constant(5.0))

<tf.Tensor: shape=(), dtype=float32, numpy=30.0>

In [6]:
# All trainable variables
print("trainable variables:", simple_module.trainable_variables)
# Every variable
print("all variables:", simple_module.variables)


trainable variables: (<tf.Variable 'train_me:0' shape=() dtype=float32, numpy=5.0>,)
all variables: (<tf.Variable 'train_me:0' shape=() dtype=float32, numpy=5.0>, <tf.Variable 'do_not_train_me:0' shape=() dtype=float32, numpy=5.0>)


This is an example of a two-layer linear layer model made out of modules.

First a dense (linear) layer:

In [15]:
class Dense(tf.Module):
  def __init__(self, in_features, out_features, name=None):
    super().__init__(name=name)
    self.w = tf.Variable(
      tf.random.normal([in_features, out_features]), name='w')
    self.b = tf.Variable(tf.zeros([out_features]), name='b')
  def __call__(self, x):
    y = tf.matmul(x, self.w) + self.b
    return tf.nn.relu(y)

And then the complete model, which makes two layer instances and applies them:

In [18]:
class SequentialModule(tf.Module):
  def __init__(self, name=None):
    super().__init__(name=name)

    self.dense_1 = Dense(in_features=3, out_features=3)
    self.dense_2 = Dense(in_features=3, out_features=2)

  def __call__(self, x):
    x = self.dense_1(x)
    return self.dense_2(x)

# You have made a model!
my_model = SequentialModule(name="the_model")

# Call it, with random results
print("Model results:", my_model(tf.constant([[2.0, 2.0, 2.0]])))

Model results: tf.Tensor([[3.018578  7.6579947]], shape=(1, 2), dtype=float32)


In [22]:
print("Submodules:", my_model.submodules)


Submodules: (<__main__.Dense object at 0x7f5c0be86940>, <__main__.Dense object at 0x7f5c0ba5df40>)


In [23]:
for var in my_model.variables:
  print(var, "\n")

<tf.Variable 'b:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)> 

<tf.Variable 'w:0' shape=(3, 3) dtype=float32, numpy=
array([[-0.49524412,  0.5959003 , -1.1662264 ],
       [ 1.1274235 ,  1.6603383 ,  0.76173085],
       [ 0.7050056 , -0.2426442 , -0.95915985]], dtype=float32)> 

<tf.Variable 'b:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)> 

<tf.Variable 'w:0' shape=(3, 2) dtype=float32, numpy=
array([[ 1.3131576 ,  1.9261199 ],
       [-0.12249015,  0.6224784 ],
       [ 0.4980017 ,  0.82830286]], dtype=float32)> 



### Waiting to create variables


In [24]:
class FlexibleDenseModule(tf.Module):
  # Note: No need for `in_features`
  def __init__(self, out_features, name=None):
    super().__init__(name=name)
    self.is_built = False
    self.out_features = out_features

  def __call__(self, x):
    # Create variables on first call.
    if not self.is_built:
      self.w = tf.Variable(
        tf.random.normal([x.shape[-1], self.out_features]), name='w')
      self.b = tf.Variable(tf.zeros([self.out_features]), name='b')
      self.is_built = True

    y = tf.matmul(x, self.w) + self.b
    return tf.nn.relu(y)

In [34]:
# Used in a module
class MySequentialModule(tf.Module):
  def __init__(self, name=None):
    super().__init__(name=name)

    self.dense_1 = FlexibleDenseModule(out_features=3)
    self.dense_2 = FlexibleDenseModule(out_features=2)

  def __call__(self, x):
    x = self.dense_1(x)
    return self.dense_2(x)

my_model = MySequentialModule(name="the_model")
print("Model results:", my_model(tf.constant([[2.0, 2.0, 2.0]])))

Model results: tf.Tensor([[2.2296543 0.       ]], shape=(1, 2), dtype=float32)


saving weights

In [35]:
chkp_path = "my_checkpoint"
checkpoint = tf.train.Checkpoint(model=my_model)
checkpoint.write(chkp_path)

'my_checkpoint'

In [36]:
!ls my_checkpoint*

my_checkpoint.data-00000-of-00001  my_checkpoint.index


looking inside thte checkpoint by using `list_variables`

In [37]:
tf.train.list_variables(chkp_path)

[('_CHECKPOINTABLE_OBJECT_GRAPH', []),
 ('model/dense_1/b/.ATTRIBUTES/VARIABLE_VALUE', [3]),
 ('model/dense_1/w/.ATTRIBUTES/VARIABLE_VALUE', [3, 3]),
 ('model/dense_2/b/.ATTRIBUTES/VARIABLE_VALUE', [2]),
 ('model/dense_2/w/.ATTRIBUTES/VARIABLE_VALUE', [3, 2])]

reloading the model


In [38]:
new_model = MySequentialModule()
new_checkpoint = tf.train.Checkpoint(model=new_model)
new_checkpoint.restore("my_checkpoint")

# Should be the same result as above
new_model(tf.constant([[2.0, 2.0, 2.0]]))

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[2.2296543, 0.       ]], dtype=float32)>

## Saving functions


defining a graph in the model above by adding the `@tf.function` decorator to indicate that this code should run as a graph.

In [39]:
class MySequentialModule(tf.Module):
  def __init__(self, name=None):
    super().__init__(name=name)

    self.dense_1 = Dense(in_features=3, out_features=3)
    self.dense_2 = Dense(in_features=3, out_features=2)

  @tf.function
  def __call__(self, x):
    x = self.dense_1(x)
    return self.dense_2(x)

# You have made a model with a graph!
my_model = MySequentialModule(name="the_model")

In [44]:
print(my_model([[2.0, 2.0, 2.0]]))
print(my_model([[[2.0, 2.0, 2.0], [2.0, 2.0, 2.0]]]))

tf.Tensor([[0.        0.9102193]], shape=(1, 2), dtype=float32)
tf.Tensor(
[[[0.        0.9102193]
  [0.        0.9102193]]], shape=(1, 2, 2), dtype=float32)


You can visualize the graph by tracing it within a TensorBoard summary.

In [43]:
# Set up logging.
stamp = datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = "logs/func/%s" % stamp
writer = tf.summary.create_file_writer(logdir)

# Create a new model to get a fresh trace
# Otherwise the summary will not see the graph.
new_model = MySequentialModule()

# Bracket the function call with
# tf.summary.trace_on() and tf.summary.trace_export().
tf.summary.trace_on(graph=True)
tf.profiler.experimental.start(logdir)
# Call only one tf.function when tracing.
z = print(new_model(tf.constant([[2.0, 2.0, 2.0]])))
with writer.as_default():
  tf.summary.trace_export(
      name="my_func_trace",
      step=0,
      profiler_outdir=logdir)

AlreadyExistsError: ignored

Launch TensorBoard to view the resulting trace:

![A screenshot of the graph in TensorBoard](https://github.com/tensorflow/docs/blob/master/site/en/guide/images/tensorboard_graph.png?raw=1)

### Creating a `SavedModel`

The recommended way of sharing completely trained models is to use `SavedModel`.  `SavedModel` contains both a collection of functions and a collection of weights. 

You can save the model you have just trained as follows:

In [47]:
tf.saved_model.save(my_model, "the_saved_model")

In [48]:
# Inspect the SavedModel in the directory
!ls -l the_saved_model

total 32
drwxr-xr-x 2 root root  4096 Apr  7 05:13 assets
-rw-r--r-- 1 root root    56 Apr  7 05:13 fingerprint.pb
-rw-r--r-- 1 root root 17110 Apr  7 05:13 saved_model.pb
drwxr-xr-x 2 root root  4096 Apr  7 05:13 variables


In [50]:
# The variables/ directory contains a checkpoint of the variables 
!ls -l the_saved_model/variables

total 8
-rw-r--r-- 1 root root 490 Apr  7 05:13 variables.data-00000-of-00001
-rw-r--r-- 1 root root 356 Apr  7 05:13 variables.index


In [51]:
new_model = tf.saved_model.load("the_saved_model")

In [52]:
isinstance(new_model, SequentialModule)

False

In [55]:
print(my_model([[2.3, 2.0, 2.0]]))
print(my_model([[[2.0, 2.0, 2.0], [2.0, 2.0, 2.0]]]))

tf.Tensor([[0.        0.6794564]], shape=(1, 2), dtype=float32)
tf.Tensor(
[[[0.        0.9102193]
  [0.        0.9102193]]], shape=(1, 2, 2), dtype=float32)


In [56]:
class MyDense(tf.keras.layers.Layer):
  # Adding **kwargs to support base Keras layer arguments
  def __init__(self, in_features, out_features, **kwargs):
    super().__init__(**kwargs)

    # This will soon move to the build step; see below
    self.w = tf.Variable(
      tf.random.normal([in_features, out_features]), name='w')
    self.b = tf.Variable(tf.zeros([out_features]), name='b')
  def call(self, x):
    y = tf.matmul(x, self.w) + self.b
    return tf.nn.relu(y)

simple_layer = MyDense(name="simple", in_features=3, out_features=3)

In [57]:
simple_layer([[2.0, 2.0, 2.0]])

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[0.      , 0.      , 5.927107]], dtype=float32)>

### The `build` 



In [58]:
class FlexibleDense(tf.keras.layers.Layer):
  # Note the added `**kwargs`, as Keras supports many arguments
  def __init__(self, out_features, **kwargs):
    super().__init__(**kwargs)
    self.out_features = out_features

  def build(self, input_shape):  # Create the state of the layer (weights)
    self.w = tf.Variable(
      tf.random.normal([input_shape[-1], self.out_features]), name='w')
    self.b = tf.Variable(tf.zeros([self.out_features]), name='b')

  def call(self, inputs):  # Defines the computation from inputs to outputs
    return tf.matmul(inputs, self.w) + self.b

# Create the instance of the layer
flexible_dense = FlexibleDense(out_features=3)

At this point, the model has not been built, so there are no variables:

In [59]:
flexible_dense.variables

[]

Calling the function allocates appropriately-sized variables:

In [60]:
# Call it, with predictably random results
print("Model results:", flexible_dense(tf.constant([[2.0, 2.0, 2.0], [3.0, 3.0, 3.0]])))

Model results: tf.Tensor(
[[ 4.540904  -2.387653   1.0209712]
 [ 6.8113565 -3.5814798  1.531457 ]], shape=(2, 3), dtype=float32)


In [61]:
flexible_dense.variables

[<tf.Variable 'flexible_dense/w:0' shape=(3, 3) dtype=float32, numpy=
 array([[ 0.09361359, -0.76879007, -0.49067268],
        [-0.25557682,  0.8658152 ,  0.8930336 ],
        [ 2.4324152 , -1.2908517 ,  0.10812463]], dtype=float32)>,
 <tf.Variable 'flexible_dense/b:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>]

In [62]:
try:
  print("Model results:", flexible_dense(tf.constant([[2.0, 2.0, 2.0, 2.0]])))
except tf.errors.InvalidArgumentError as e:
  print("Failed:", e)

Failed: Exception encountered when calling layer 'flexible_dense' (type FlexibleDense).

{{function_node __wrapped__MatMul_device_/job:localhost/replica:0/task:0/device:CPU:0}} Matrix size-incompatible: In[0]: [1,4], In[1]: [3,3] [Op:MatMul]

Call arguments received by layer 'flexible_dense' (type FlexibleDense):
  • inputs=tf.Tensor(shape=(1, 4), dtype=float32)


In [63]:
class MySequentialModel(tf.keras.Model):
  def __init__(self, name=None, **kwargs):
    super().__init__(**kwargs)

    self.dense_1 = FlexibleDense(out_features=3)
    self.dense_2 = FlexibleDense(out_features=2)
  def call(self, x):
    x = self.dense_1(x)
    return self.dense_2(x)

# You have made a Keras model!
my_sequential_model = MySequentialModel(name="the_model")

# Call it on a tensor, with random results
print("Model results:", my_sequential_model(tf.constant([[2.0, 2.0, 2.0]])))


Model results: tf.Tensor([[-2.4241824 -2.5308843]], shape=(1, 2), dtype=float32)


In [64]:
my_sequential_model.variables

[<tf.Variable 'my_sequential_model/flexible_dense_1/w:0' shape=(3, 3) dtype=float32, numpy=
 array([[-0.59814364, -1.855396  ,  0.00801724],
        [-0.07100037, -0.33233657,  0.76396936],
        [ 1.936796  , -0.48694336, -0.7854162 ]], dtype=float32)>,
 <tf.Variable 'my_sequential_model/flexible_dense_1/b:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>,
 <tf.Variable 'my_sequential_model/flexible_dense_2/w:0' shape=(3, 2) dtype=float32, numpy=
 array([[ 0.37418702, -0.6783775 ],
        [ 0.6295522 ,  0.15461572],
        [ 0.19225816, -0.5995299 ]], dtype=float32)>,
 <tf.Variable 'my_sequential_model/flexible_dense_2/b:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>]

In [65]:
my_sequential_model.submodules

(<__main__.FlexibleDense at 0x7f5c0b9a8eb0>,
 <__main__.FlexibleDense at 0x7f5c0bb16550>)

In [66]:
inputs = tf.keras.Input(shape=[3,])

x = FlexibleDense(3)(inputs)
x = FlexibleDense(2)(x)

my_functional_model = tf.keras.Model(inputs=inputs, outputs=x)

my_functional_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3)]               0         
                                                                 
 flexible_dense_3 (FlexibleD  (None, 3)                12        
 ense)                                                           
                                                                 
 flexible_dense_4 (FlexibleD  (None, 2)                8         
 ense)                                                           
                                                                 
Total params: 20
Trainable params: 20
Non-trainable params: 0
_________________________________________________________________


In [67]:
my_functional_model(tf.constant([[2.0, 2.0, 2.0]]))

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[-0.25973415,  0.37283826]], dtype=float32)>

## Saving Keras model


In [69]:
my_sequential_model.save("exname_of_file")

In [70]:
reconstructed_model = tf.keras.models.load_model("exname_of_file")

In [71]:
reconstructed_model(tf.constant([[2.0, 2.0, 2.0]]))

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[-2.4241824, -2.5308843]], dtype=float32)>